In [1]:
import pandas as pd
import numpy as np
import re
import csv
import string

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import torch

np.random.seed(500)

c:\users\harim\appdata\local\programs\python\python36\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_prescriptions = pd.read_csv('C:/Users/harim/OneDrive/Documents/GitHub/dl4h_final_project_1/data/processed/ndc_codes_extracted.csv', dtype=str)

df_prescriptions.head()

,Unnamed: 0,NDC,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC
0,0,00469061711,Tacrolimus,Tacrolimus,Tacrolimus
1,1,00056017275,Warfarin,Warfarin,Warfarin
2,2,00338055002,Heparin Sodium,UNKNOWN,UNKNOWN
3,4,00054829725,Furosemide,Furosemide,Furosemide
4,5,00056016975,Warfarin,Warfarin,Warfarin


In [3]:

# ndc_codes = set()

# for index, record in df_prescriptions.iterrows():
#     ndc_codes.add(record['NDC'].strip())
        
# ndc_codes = list(ndc_codes)

# print(len(ndc_codes))

In [4]:
df = pd.read_csv('../data/processed/NOTE_EVENTS_WITH_DRUGS.csv', dtype=str)

df = df.head(1000)

In [5]:
df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT,CLEAN_TEXT,DRUGS_PRESENT
0,174,22532,167853,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...,admission date 2151716 discharge date 215184 s...,NaN
1,175,13702,107527,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...,admission date 211862 discharge date 2118614 d...,"00409471332,00548301200,00409471332,0054830120..."
2,176,13702,167118,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...,admission date 211954 discharge date 2119525 s...,"00069153041,51079045120,59762153005,0007101554..."
3,177,13702,196489,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...,admission date 2124721 discharge date 2124818 ...,"00069153041,51079045120,59762153005,0003412008..."
4,178,26880,135453,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...,admission date 216233 discharge date 2162325 d...,"51079055271,00245010450,00536338101,0057470505..."


In [6]:
df['DRUGS_PRESENT_SPLIT'] = df['DRUGS_PRESENT'].str.split(pat=',') #.apply(lambda x: x.split(','))

df = df[df['DRUGS_PRESENT_SPLIT'].notnull()]

print("Count After Null Filter:", df.count())

Count After Null Filter: ROW_ID                 979
SUBJECT_ID             979
HADM_ID                979
CHARTDATE              979
CHARTTIME                0
STORETIME                0
CATEGORY               979
DESCRIPTION            979
CGID                     0
ISERROR                  0
TEXT                   979
CLEAN_TEXT             979
DRUGS_PRESENT          979
DRUGS_PRESENT_SPLIT    979
dtype: int64


In [7]:

free_text = df['CLEAN_TEXT']
label_codes = df['DRUGS_PRESENT_SPLIT']

### Text Vectorization

In [8]:

text_vectorizer = TfidfVectorizer(ngram_range=(3, 3),
                             analyzer='char')
X = text_vectorizer.fit_transform(free_text)

In [9]:
print(X.shape, text_vectorizer.get_feature_names())
print("....", X[0,0])

(979, 14896) [' 0 ', ' 00', ' 01', ' 02', ' 03', ' 04', ' 05', ' 06', ' 07', ' 08', ' 09', ' 0m', ' 0n', ' 0r', ' 1 ', ' 10', ' 11', ' 12', ' 13', ' 14', ' 15', ' 16', ' 17', ' 18', ' 19', ' 1a', ' 1b', ' 1c', ' 1d', ' 1e', ' 1g', ' 1h', ' 1i', ' 1l', ' 1m', ' 1n', ' 1o', ' 1p', ' 1r', ' 1s', ' 1t', ' 1u', ' 1v', ' 1w', ' 1x', ' 1y', ' 2 ', ' 20', ' 21', ' 22', ' 23', ' 24', ' 25', ' 26', ' 27', ' 28', ' 29', ' 2a', ' 2b', ' 2c', ' 2d', ' 2e', ' 2g', ' 2h', ' 2i', ' 2l', ' 2m', ' 2n', ' 2o', ' 2p', ' 2q', ' 2r', ' 2s', ' 2t', ' 2u', ' 2v', ' 2w', ' 2x', ' 2y', ' 3 ', ' 30', ' 31', ' 32', ' 33', ' 34', ' 35', ' 36', ' 37', ' 38', ' 39', ' 3a', ' 3b', ' 3c', ' 3d', ' 3e', ' 3f', ' 3g', ' 3h', ' 3l', ' 3m', ' 3o', ' 3p', ' 3r', ' 3s', ' 3t', ' 3u', ' 3v', ' 3w', ' 3x', ' 3y', ' 4 ', ' 40', ' 41', ' 42', ' 43', ' 44', ' 45', ' 46', ' 47', ' 48', ' 49', ' 4a', ' 4b', ' 4c', ' 4d', ' 4e', ' 4f', ' 4g', ' 4h', ' 4i', ' 4l', ' 4m', ' 4o', ' 4p', ' 4r', ' 4s', ' 4t', ' 4u', ' 4v', ' 4w', ' 4x',

### Label Encoding

In [10]:
Y = np.array(label_codes)



In [11]:
len(set(Y[0]))

81

In [12]:
from sklearn.preprocessing import MultiLabelBinarizer
    
lb = MultiLabelBinarizer()
Y = lb.fit_transform(Y)

# Y = Y.T

In [13]:
print("Labels found in first example: ", Y[0].sum())

print(Y[0])

Labels found in first example:  81
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 0 0 0 1 0 0 0 0 1 1 1 0 1 1
 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 1 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 1
 1 0 1 0 0 0 0 0 0 0 1 1 0 1 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 1 0 0 1 0 1 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0
 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0]


In [14]:
print("X: ", X.shape, " Y: ", Y.shape)

X:  (979, 14896)  Y:  (979, 427)


In [15]:
train_samples, no_of_featres = X.shape
ignore, label_count = Y.shape

print(train_samples, no_of_featres, label_count)

979 14896 427


In [212]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=109)

In [213]:
def collate_fn(data):
    
#     print("collate.....", len(data))
#     print("collate.....", data[0].shape)
    
    data = np.array(data).reshape((len(data), data[0].shape[0], data[0].shape[1]))
    
    res = torch.tensor(data)
    
    print("collate.....", res.dtype, res.shape)
    return res

def load_data(train_dataset, val_dataset):
    
    batch_size = 32
    # your code here
    train_loader = DataLoader(train_dataset.todense(), batch_size = 64, shuffle = True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset.todense(), batch_size = 64, shuffle = False, collate_fn=collate_fn)
    
    return train_loader, val_loader



In [214]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

class Network(nn.Module):

    def __init__(self):
        super().__init__()

        self.fc1 = nn.LSTM(no_of_featres, 1000, 1)
        self.a1 = nn.ReLU()
        self.fc2 = nn.Linear(1000, label_count)

    def forward(self,x):
        print(".....", x[0].dtype)
        
        x = self.fc1(x)
        x = self.a1(x)
        x = self.fc2(x)

        return x

In [215]:
print(torch.cuda.is_available())

False


In [216]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {device} device", device)

from torch.optim import Adam

model = Network()

criterion = nn.MSELoss()
EPOCHS = 10
optimizer = Adam(model.parameters(), lr = 0.001)


Using {device} device cpu


In [217]:

train_loader, val_loader = load_data(X_train, X_test)

In [218]:
def train(model, train_loader, val_loader, n_epochs):    
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        for x in train_loader:
            print("Batch..", x.dtype)
            
            loss = None
            
            optimizer.zero_grad()
            y_hat = model(x)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        train_loss = train_loss / len(train_loader)
        print("Epoch: ", epoch, "Loss: ", train_loss)
#         p, r, f, roc_auc = eval_model(model, val_loader)

In [219]:
train(model, train_loader, val_loader, 5)

collate..... torch.int64 torch.Size([64, 1, 14896])
Batch.. torch.int64
..... torch.int64


RuntimeError: expected scalar type Long but found Float

In [197]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred, average='micro'))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred, average='micro'))

NameError: name 'y_pred' is not defined

In [ ]:
import pickle
with open('C:\Users\harim\OneDrive\Documents\GitHub\dl4h_final_project_1\trained_models\vectorizer.pk', 'wb') as fin:
    pickle.dump(text_vectorizer, fin)

In [ ]:
output = open('C:\Users\harim\OneDrive\Documents\GitHub\dl4h_final_project_1\trained_models\label_encoder.pkl', 'wb')
pickle.dump(lb, output)
output.close()

In [ ]:
from joblib import dump, load

dump(model, 'C:\Users\harim\OneDrive\Documents\GitHub\dl4h_final_project_1\trained_models\model.joblib') 